Instaalling and Importing

In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
import os
import re
import spacy
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from transformers import DistilBertTokenizer, DistilBertModel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load Pretrained Model

In [ ]:

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

Tokenizing and Embedding

In [ ]:
import torch
def get_embeddings(text):

    tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**tokens)
        embeddings = outputs.last_hidden_state.mean(dim=1)

    return embeddings


Loading Extracted Data

In [ ]:
df = pd.read_csv('/content/extracted.csv')

In [ ]:
df

,Unnamed: 0,File,Category,skills,education
0,0,12587973.pdf,ARTS,experience nsubstitute teacher skills appropri...,university city state nbachelor arts
1,1,10527994.pdf,ARTS,skills noutlook excel word powerpoint quickboo...,university
2,2,12467531.pdf,ARTS,skills knowledge experiences teacher advance n...,university california degree elementary educat...
3,3,11336022.pdf,ARTS,skills work ethic achievement asset company ex...,college city state teachers education nhigh sc...
4,4,10504237.pdf,HR,experience n skills nanatomy aseptic technique...,university washington university school educat...
...,...,...,...,...,...
2493,2493,74126637.pdf,ADVOCATE,skills report assessment writing nexperienced ...,university
2494,2494,54067174.pdf,ADVOCATE,experience ninbound outbound blended environme...,NaN
2495,2495,95714702.pdf,ADVOCATE,experience variety areas including human resou...,university university phoenix
2496,2496,74191424.pdf,ADVOCATE,skills growing company skill best possible way...,university university


In [ ]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df

,File,Category,skills,education
0,12587973.pdf,ARTS,experience nsubstitute teacher skills appropri...,university city state nbachelor arts
1,10527994.pdf,ARTS,skills noutlook excel word powerpoint quickboo...,university
2,12467531.pdf,ARTS,skills knowledge experiences teacher advance n...,university california degree elementary educat...
3,11336022.pdf,ARTS,skills work ethic achievement asset company ex...,college city state teachers education nhigh sc...
4,10504237.pdf,HR,experience n skills nanatomy aseptic technique...,university washington university school educat...
...,...,...,...,...
2493,74126637.pdf,ADVOCATE,skills report assessment writing nexperienced ...,university
2494,54067174.pdf,ADVOCATE,experience ninbound outbound blended environme...,NaN
2495,95714702.pdf,ADVOCATE,experience variety areas including human resou...,university university phoenix
2496,74191424.pdf,ADVOCATE,skills growing company skill best possible way...,university university


Loading Job_Descriptions

In [ ]:
# Load job descriptions (you can fetch them from the provided dataset)
# Replace this with the actual job descriptions you want to use
jd_df=pd.read_csv('/content/drive/MyDrive/AIML/training_data.csv')

In [ ]:
jd_df['job_description']

0      minimum qualifications\nbachelors degree or eq...
1      description\nas an asc you will be highly infl...
2      its an amazing time to be joining netflix as w...
3      description\n\nweb designers looking to expand...
4      at trackfive weve got big goals were on a miss...
                             ...                        
848    job description\n\nparttime\n\nmake big money ...
849    responsibilities\nparkers internship program w...
850     the borgen project is an innovative national ...
851    put the world on vacation\n\nat wyndham destin...
852    this job handles customer inquiries by telepho...
Name: job_description, Length: 853, dtype: object

In [ ]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Preprocessing Job_Descriptions

In [ ]:
from nltk.corpus import stopwords
def preprocess(txt):
    # convert all characters in the string to lower case
    #txt = txt.lower()
    txt = txt.lower() if isinstance(txt, str) else ' '.join(txt).lower()
    # remove non-english characters, punctuation and numbers
    txt = re.sub('[^a-zA-Z]', ' ', txt)
    txt = re.sub('http\S+\s*', ' ', txt)  # remove URLs
    txt = re.sub('RT|cc', ' ', txt)  # remove RT and cc
    txt = re.sub('#\S+', '', txt)  # remove hashtags
    txt = re.sub('@\S+', '  ', txt)  # remove mentions
    txt = re.sub('\s+', ' ', txt)  # remove extra whitespace
    #txt = [w for w in txt if w != "skills"]
    # tokenize word
    txt = nltk.tokenize.word_tokenize(txt)
    # remove stop words
    txt = [w for w in txt if not w in nltk.corpus.stopwords.words('english')]


    return ' '.join(txt)

In [ ]:
# preprocessing text
jd_df['job_description'] = jd_df['job_description'].apply(lambda w: preprocess(w))

In [ ]:
jd_df1=jd_df.head(15)

In [ ]:
jd_df1['job_description']

0     minimum qualifications bachelors degree equiva...
1     description asc highly influential growing min...
2     amazing time joining netflix continue transfor...
3     description web designers looking expand profe...
4     trackfive weve got big goals mission revolutio...
5     designups nashville based design interactive a...
6     position web designer responsible providing de...
7     job description zander insurance group one lar...
8     tuff growth marketing team working clients dri...
9     type requisition regular clearance level must ...
10    web developer job id positions location ustnna...
11    snapshot looking back end developer join organ...
12    seeking senior ui designer relishes methodical...
13    design develop test high quality software feat...
14    youre passionate building better future indivi...
Name: job_description, dtype: object

Job_Description Embeddings

In [ ]:
# Create embeddings for job descriptions
job_description_embeddings = [get_embeddings(desc) for desc in jd_df1['job_description']]


In [ ]:
job_description_embeddings

[tensor([[-1.1387e-01,  3.6822e-01,  5.8116e-01, -1.4168e-01,  3.4855e-01,
           3.2927e-02,  3.8141e-01, -7.4241e-02,  1.1353e-02, -2.0594e-01,
           5.5454e-02, -1.0704e-01, -9.0450e-04,  1.1748e-01, -7.1928e-02,
           3.2097e-01, -7.9320e-02,  1.7953e-01, -4.1336e-02,  4.5536e-01,
           3.2330e-01,  1.6798e-01,  1.4112e-01,  4.0250e-01,  3.5909e-01,
           8.3329e-02,  3.3981e-02, -1.5256e-01,  1.4375e-01, -3.4934e-02,
           1.6255e-01, -4.8891e-02,  2.1291e-01, -1.2509e-01, -1.1628e-01,
          -6.2486e-02,  1.4127e-02, -2.2881e-01,  4.5192e-02,  1.7842e-01,
          -4.8147e-01, -3.5077e-01, -5.8466e-02,  1.7050e-01, -2.1165e-01,
          -2.5183e-01, -5.2709e-01, -6.5600e-03,  4.1786e-02, -3.2287e-01,
          -4.4791e-01,  1.3864e-01,  6.4769e-02, -9.3804e-02,  2.1102e-02,
           4.6140e-01,  1.1535e-01, -2.3652e-01, -5.5457e-01, -4.5819e-01,
          -4.0698e-02, -1.2452e-01, -1.4764e-01, -1.4994e-01, -9.0174e-02,
           3.9716e-02,  1

CV embeddings and CV-JD similarities

In [ ]:

cv_job_similarity = {}

# Calculate cosine similarity for each job description and CV
for idx, job_embedding in enumerate(job_description_embeddings):
    for cv_idx, cv_row in df.iterrows():
        cv_embedding = get_embeddings(str(cv_row['skills']) + ' ' + str(cv_row['education']))
        similarity_score = cosine_similarity(job_embedding, cv_embedding)

        # Store the similarity score in the dictionary
        if idx not in cv_job_similarity:
            cv_job_similarity[idx] = []
        cv_job_similarity[idx].append((cv_row['File'], similarity_score))

In [ ]:
cv_job_similarity

{0: [('12587973.pdf', array([[0.8784154]], dtype=float32)),
  ('10527994.pdf', array([[0.7795436]], dtype=float32)),
  ('12467531.pdf', array([[0.85271657]], dtype=float32)),
  ('11336022.pdf', array([[0.8051798]], dtype=float32)),
  ('10504237.pdf', array([[0.7738783]], dtype=float32)),
  ('11943065.pdf', array([[0.7976852]], dtype=float32)),
  ('11616482.pdf', array([[0.8419587]], dtype=float32)),
  ('10909720.pdf', array([[0.75361526]], dtype=float32)),
  ('13087952.pdf', array([[0.8496923]], dtype=float32)),
  ('16820422.pdf', array([[0.860387]], dtype=float32)),
  ('14958913.pdf', array([[0.8384615]], dtype=float32)),
  ('13296856.pdf', array([[0.8643041]], dtype=float32)),
  ('22408666.pdf', array([[0.8390853]], dtype=float32)),
  ('13330982.pdf', array([[0.7619203]], dtype=float32)),
  ('16270906.pdf', array([[0.8895037]], dtype=float32)),
  ('21531811.pdf', array([[0.7868483]], dtype=float32)),
  ('19464810.pdf', array([[0.7889375]], dtype=float32)),
  ('17311685.pdf', array([[

Ranking CVs

In [ ]:

# Rank CVs based on similarity for each job description
top_5_cvs = {}
for idx, similarity_scores in cv_job_similarity.items():
    sorted_cvs = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_5_cvs[idx] = sorted_cvs[:5]

# Print or save the top 5 CVs for each job description
for idx, top_cvs in top_5_cvs.items():
    print(f"Job Description {idx + 1}")
    for cv, score in top_cvs:
        print(f"CV: {cv}, Similarity Score: {score}")
    print()


Job Description 1
CV: 16276121.pdf, Similarity Score: [[0.92162263]]
CV: 28609364.pdf, Similarity Score: [[0.921613]]
CV: 91318828.pdf, Similarity Score: [[0.90523714]]
CV: 28679359.pdf, Similarity Score: [[0.8992003]]
CV: 25038571.pdf, Similarity Score: [[0.89820355]]

Job Description 2
CV: 16276121.pdf, Similarity Score: [[0.92739487]]
CV: 18525641.pdf, Similarity Score: [[0.92561245]]
CV: 80679862.pdf, Similarity Score: [[0.9152492]]
CV: 18927233.pdf, Similarity Score: [[0.91468376]]
CV: 81508860.pdf, Similarity Score: [[0.9044921]]

Job Description 3
CV: 16276121.pdf, Similarity Score: [[0.91210306]]
CV: 91318828.pdf, Similarity Score: [[0.90935683]]
CV: 23085604.pdf, Similarity Score: [[0.90932834]]
CV: 28609364.pdf, Similarity Score: [[0.9069791]]
CV: 27419236.pdf, Similarity Score: [[0.89313006]]

Job Description 4
CV: 16276121.pdf, Similarity Score: [[0.9349415]]
CV: 91318828.pdf, Similarity Score: [[0.931939]]
CV: 27419236.pdf, Similarity Score: [[0.9212103]]
CV: 19053815.pdf,